# Your Title Information Goes Here
Fill me in

## Setup

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `C:\Users\Lillian K\OneDrive\Documents\CHEME 5660\5660-FINAL-PUBLIC-REPO\CHEME-5660-Final-Project\Project.toml`
    Updating `C:\Users\Lillian K\OneDrive\Documents\CHEME 5660\5660-FINAL-PUBLIC-REPO\CHEME-5660-Final-Project\Manifest.toml`
  [79e6a3ab] ↑ Adapt v3.7.1 ⇒ v3.7.2
  [7a1cc6ca] ↑ FFTW v1.7.1 ⇒ v1.7.2
  [1a297f60] ↑ FillArrays v1.9.2 ⇒ v1.9.3
  [5c8ed15e] ↑ LinearOperators v2.5.2 ⇒ v2.6.0
  [b8a86587] ↑ NearestNeighbors v0.4.13 ⇒ v0.4.15
  [90014a1f] ↑ PDMats v0.11.30 ⇒ v0.11.31
  [90137ffa] ↑ StaticArrays v1.7.0 ⇒ v1.8.0
  [aacddb02] ↑ JpegTurbo_jll v2.1.91+0 ⇒ v3.0.1+0
  [856f044c] ↑ MKL_jll v2023.2.0+0 ⇒ v2024.0.0+0
  [b53b4c65] ↑ libpng_jll v1.6.38+1 ⇒ v1.6.40+0
Precompiling project...
  ✓ Distributions
  ✓ Distributions → DistributionsTestExt
  ✓ Distributions → DistributionsChainRulesCoreExt
  ✓ LsqFit
  ✓ KernelDensity
  ✓ StatsPlots
 

## Prerequisites: Load historical dataset, compute expected returns and get 2023 `SPY` data
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) since `01-03-2018` until `12-01-2023`, along with data for a few exchange traded funds and volatility products during that time. 

In [2]:
original_dataset = load(joinpath(_PATH_TO_DATA, 
        "SP500-Daily-OHLC-1-3-2018-to-12-01-2023.jld2")) |> x-> x["dataset"];

### Clean the data
Not all of the tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquistion or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximim value, e.g., `AAPL` and save that value in the `maximum_number_trading_days` variable:

In [3]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [4]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

Let's get a list of firms that we have in cleaned up `dataset`, and save it in the `all_tickers` array:

In [5]:
all_tickers = keys(dataset) |> collect |> sort;
K = length(all_tickers);

### Get the 2023 `SPY` data

In [6]:
startdate = Date(2023,01,03);
SPY_dataset = dataset["SPY"];
SPY_df = filter(:timestamp => x-> x >= startdate, SPY_dataset);

### Compute the expected return for all firms in the dataset
The expected return $\mathbb{E}(r_{i})$ and covariance matrix $\Sigma$ will be used in our calculations, so we'll provide values for both of these items for the entire data set (all `N = 459` tickers), and then you can pick out which tickers you are interested in. 

* First, we compute the expected (annualized) log return by passing the `dataset` and the entire list of firms we have in the dataset (held in the $N\times{1}$ `all_array` array) to the `log_return_matrix(...)` method. The result is stored in the `all_firms_return_matrix` variable, a $T-1\times{N}$ array of log return values. Each row of `all_firms_return_matrix` corresponds to a time-value, while each column corresponds to a firm:

In [7]:
all_firms_return_matrix = log_return_matrix(dataset, all_tickers, 
    Δt = (1.0/252.0), risk_free_rate = 0.0);

## Your project starts here ....

In [8]:
all_tickers = keys(dataset) |> collect |> sort;

## Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

In [9]:
risk_free_rate = 0.05;

In [10]:
all_firms_excess_return_matrix = log_return_matrix(dataset, all_tickers, 
    Δt = (1.0/252.0), risk_free_rate = risk_free_rate);

In [11]:
μ = mean(all_firms_excess_return_matrix, dims=1) |> vec;

LoadError: ArgumentError: No file exists at given path: C:\Users\Lillian K\OneDrive\Documents\CHEME 5660\5660-FINAL-PUBLIC-REPO\CHEME-5660-Final-Project\data\SIMs-NewCO-PD1-CHEME-5660-Fall-2023.jld2